# Indexation hierarchique

L'indexation hierarchique est une fonctionnalité importante de pandas qui permet d'avoir plusieurs niveaux d'indexes sur un axe. 

In [4]:
import numpy as np
import pandas as pd

In [6]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])  

In [7]:
data

a  1   -0.533840
   2   -0.487353
   3    0.143877
b  1    1.199166
   3   -1.021224
c  1   -1.578583
   2    0.710838
d  2   -0.767163
   3   -0.900530
dtype: float64

In [8]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [9]:
data['b']

1    1.199166
3   -1.021224
dtype: float64

In [10]:
data['b':'c']

b  1    1.199166
   3   -1.021224
c  1   -1.578583
   2    0.710838
dtype: float64

In [11]:
data.loc[['b', 'd']]

b  1    1.199166
   3   -1.021224
d  2   -0.767163
   3   -0.900530
dtype: float64

On peut utiliser un niveau interne lors de la séléction:

In [12]:
data.loc[:, 2]

a   -0.487353
c    0.710838
d   -0.767163
dtype: float64

L'indexation hierarchique joue un rôle primordial pour le changement du format de données et les opérations d'aggrégation comme le pivotage.

In [13]:
data.unstack()

,1,2,3
a,-0.533840,-0.487353,0.143877
b,1.199166,NaN,-1.021224
c,-1.578583,0.710838,NaN
d,NaN,-0.767163,-0.900530


L'opération inverse est stack:

In [14]:
data.unstack().stack()

a  1   -0.533840
   2   -0.487353
   3    0.143877
b  1    1.199166
   3   -1.021224
c  1   -1.578583
   2    0.710838
d  2   -0.767163
   3   -0.900530
dtype: float64

Pour un DataFrame, les deux axes peuvent avoir des indexes hierarchiques:

In [15]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

In [16]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [19]:
frame.index.names = ['key1', 'key2']

In [20]:
frame.columns.names = ['state', 'color']

In [21]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

On peut créer également une MultiIndex à partir d'un tableau:

In [22]:
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

NameError: ignored

**Tri des niveaux**

La fonction **swaplevel** permet d'échanger deux niveaux ou de trier les données par valeur selon un axe.

In [23]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

La fonction **sort_index** permet de trier les données en utilisant les valeurs dans un seul niveau.

In [24]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [25]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

**Statistiques par niveau**

Plusieurs fonctions statistiques ont un argument pour spécifier le niveau qui peut être utilisé pour l'aggrégation.

In [26]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [27]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

# Fusion

# Pivotage